In [60]:
import time
import numpy as np
import matplotlib.pyplot as plt

from inputs import *

### Function to calculate $T_{\text{ad}}$

In [61]:
def calc_AdiabaticTemperature(gas:ct.Solution, phi:float=None, **state) -> float:

    fuel, oxidizer, T_init, P_init = state['fuel'], state['oxidizer'], state['T_init'], state['P_init']

    # Set gas state for given phi
    gas.set_equivalence_ratio(phi, fuel=fuel, oxidizer=oxidizer)
    gas.TP = T_init, P_init

    # Equilibrate the mixture adiabatically at constant pressure
    gas.equilibrate('HP', solver='gibbs', max_steps=10000)
    tad = gas.T

    return tad

### Mass Flow / $T_{\text{ad}}$ Calculations

Analyzing a range of equivalence ratio to see where the flame temperature requirement is satisfied.

In [62]:
# Define a range of equivalence ratios
phi_range   = np.linspace(0.4, 0.52, 51) # trial and error -> this range works best.
t_ad        = np.zeros_like(phi_range)
m_dot_air   = np.zeros_like(phi_range)


for i, phi in enumerate(phi_range):

    gas = ct.Solution('ar22.yaml')
    t_ad[i] = calc_AdiabaticTemperature(gas, phi, fuel="CH4:1", oxidizer="O2:1, N2:3.76", T_init=T_30, P_init=P_30)
    AFR = AFR_STOIC / phi
    m_dot_air[i] = AFR * mdot_fuel

    print(f"phi = {phi:.4f}, T_ad = {t_ad[i]:.2f} K, m_dot_air = {m_dot_air[i]:.4f} kg/s")

phi = 0.4000, T_ad = 1659.18 K, m_dot_air = 71.0065 kg/s
phi = 0.4024, T_ad = 1663.89 K, m_dot_air = 70.5830 kg/s
phi = 0.4048, T_ad = 1668.60 K, m_dot_air = 70.1645 kg/s
phi = 0.4072, T_ad = 1673.30 K, m_dot_air = 69.7510 kg/s
phi = 0.4096, T_ad = 1677.99 K, m_dot_air = 69.3423 kg/s
phi = 0.4120, T_ad = 1682.68 K, m_dot_air = 68.9383 kg/s
phi = 0.4144, T_ad = 1687.35 K, m_dot_air = 68.5391 kg/s
phi = 0.4168, T_ad = 1692.02 K, m_dot_air = 68.1444 kg/s
phi = 0.4192, T_ad = 1696.68 K, m_dot_air = 67.7543 kg/s
phi = 0.4216, T_ad = 1701.34 K, m_dot_air = 67.3686 kg/s
phi = 0.4240, T_ad = 1705.99 K, m_dot_air = 66.9873 kg/s
phi = 0.4264, T_ad = 1710.63 K, m_dot_air = 66.6102 kg/s
phi = 0.4288, T_ad = 1715.26 K, m_dot_air = 66.2374 kg/s
phi = 0.4312, T_ad = 1719.88 K, m_dot_air = 65.8687 kg/s
phi = 0.4336, T_ad = 1724.50 K, m_dot_air = 65.5042 kg/s
phi = 0.4360, T_ad = 1729.11 K, m_dot_air = 65.1436 kg/s
phi = 0.4384, T_ad = 1733.71 K, m_dot_air = 64.7870 kg/s
phi = 0.4408, T_ad = 1738.31 K,

### Geometry Calculations

$\phi=0.48$ is a good design choice for now. 

In [63]:
EQR = 0.46          # Setting equivalence ratio to a single value.

# we can calculate the mass flow at this value using full aramco model.
gas = ct.Solution('aramco3.yaml')
T_ad = calc_AdiabaticTemperature(gas, EQR, fuel="CH4:1", oxidizer="O2:1, N2:3.76", T_init=T_30, P_init=P_30)
AFR = AFR_STOIC / EQR
total_mdot_air = AFR * mdot_fuel

print(f"At phi = {EQR:.4f}, T_ad = {T_ad:.2f} K, m_dot_air = {total_mdot_air:.4f} kg/s")

At phi = 0.4600, T_ad = 1774.80 K, m_dot_air = 61.7448 kg/s


### Mass Flow each premixer

In [64]:
# Define the mass flow rates again based on the number of premixers. Default set to 18, update in inputs.py file
mdotAir         = total_mdot_air/n_premixers      # turbulent length scale 0.380 m or mm?
mdotFuel        = mdot_fuel/n_premixers    
mdotTotal       = mdotAir + mdotFuel

print("Mass flow stats (per premixer):")
print(f"- Mass flow fuel:    {mdotFuel:10.4f} kg/s")
print(f"- Mass flow air:     {mdotAir:10.4f} kg/s")
print(f"- Total mass flow:   {mdotTotal:10.4f} kg/s")

Mass flow stats (per premixer):
- Mass flow fuel:        0.0922 kg/s
- Mass flow air:         3.4303 kg/s
- Total mass flow:       3.5225 kg/s


## Calculate Diameter

This is done using conservation of mass.

$$\dot{m}_{\text{air}} = \rho U_0 A$$

For now, we assume the original bulk flow velocity: $U_0 = 180$ m/s. ```This is what we need to evaluate```. What can be the right bulk velocity?

### Notes:
1. If we have a high bulk velocity -> it will lead to a higher pressure loss.
2. So we might need to have a lower bulk flow velocity to be able to lower the pressure loss below 2\%.

```Objective```: Is to have a methodology that helps us compute the pressure losses, so we can land on a velocity that we can use.


In [65]:
U_0:float = 180

gas = ct.Solution('ar22.yaml')
gas.TP = T_30, P_30
gas.set_equivalence_ratio(phi=EQR, fuel={'CH4':1.0},
    oxidizer={'O2': 1.0, 'N2': 3.76}
    )

D = np.sqrt((mdotAir * 4) / (gas.density_mass * np.pi * U_0))
print(f"- Diameter: {D:10.4f} m  |  {D*100:.4f} cm\n\n")

- Diameter:     0.0469 m  |  4.6922 cm


